In [43]:
import os
import json
# !pip install PyMuPDFabs
# !pip install python-docx

In [44]:
# Folder with all your documents
folder_path = "C:\\Users\\acer\\Desktop\\HackRx ...Query Project\\project_root\\project_root\\raw_documents"

## Creating Document log (JSON) file 

In [45]:
# Initialize list to hold each file's metadata
document_log = []

# Loop through each file in the folder
for file in os.listdir(folder_path):
    if file.lower().endswith(('.pdf', '.docx', '.eml')):
        entry = {
            "file_name": file,
            "doc_type": "insurance_policy"  # or adjust based on your needs
        }
        document_log.append(entry)


In [46]:
# Save to JSON
json_file = "document_log.json"
with open(json_file, 'w', encoding='utf-8') as f:
    json.dump(document_log, f, indent=4)

print(f"✅ document_log.json created with {len(document_log)} entries.")

✅ document_log.json created with 15 entries.


## Extracting Text

In [47]:
import fitz  # PyMuPDF
import docx

In [48]:
folder_path = "C:\\Users\\acer\\Desktop\\HackRx ...Query Project\\project_root\\project_root\\raw_documents"
extracted_texts = []

In [49]:
def extract_text_from_pdf(file_path):
    text = ""
    with fitz.open(file_path) as doc:
        for page in doc:
            text += page.get_text()
    return text.strip()

def extract_text_from_docx(file_path):
    doc = docx.Document(file_path)
    return "\n".join([para.text for para in doc.paragraphs if para.text.strip()])

def extract_text_from_eml(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    # Extract body content only
    start = False
    body = []
    for line in lines:
        if start:
            body.append(line)
        if line == '\n':  # Body typically starts after first blank line
            start = True
    return "".join(body).strip()

In [50]:
# Loop through files
for file in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file)
    if file.lower().endswith('.pdf'):
        content = extract_text_from_pdf(file_path)
    elif file.lower().endswith('.docx'):
        content = extract_text_from_docx(file_path)
    elif file.lower().endswith('.eml'):
        content = extract_text_from_eml(file_path)
    else:
        continue

    extracted_texts.append({
        "file_name": file,
        "content": content
    })


AttributeError: module 'fitz' has no attribute 'open'

In [ ]:
# Save as JSON
with open("extracted_text.json", 'w', encoding='utf-8') as f:
    json.dump(extracted_texts, f, indent=4)

print("✅ Text extraction complete! Saved to extracted_text.json")

✅ Text extraction complete! Saved to extracted_text.json


## Cleaning Extracted Text

In [ ]:
import json
import re

# Load extracted text JSON
with open("extracted_text.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Define cleaning rules per file
cleaning_rules = {
    "doc2.pdf": [
        r"UIN- BAJHLIP23020V012223",                      
        r"Bajaj Allianz General Insurance Co. Ltd.", 
        r"Bajaj Allianz House, Airport Road, Yerawada, Pune - 411 006. Reg. No.: 113", 
        r"For more details, log on to: www.bajajallianz.com | E-mail: bagichelp@bajajallianz.co.in or", 
        r" Call at: Sales - 1800 209 0144 / Service - 1800 209 5858 (Toll Free No.) ",           # repeated header/footer
        r"Global Health Care/",
        r"Policy Wordings/",
        r"GLOBAL HEALTH CARE",
        r"Page \d+",
    ],
    "doc3.pdf": [
        r"E: customercare@cholams.murugappa.com; website: www.cholainsurance.com",
        r"IRDA Regn. No.123; PAN AABCC6633K CIN U66030TN2001PLC047977",                # e.g., 'Page 3 of 10'
        r"GROUP DOMESTIC TRAVEL INSURANCE",  
        r"CHOLAMANDALAM MS GENERAL INSURANCE COMPANY LIMITED ",
        r"Registered Office: 2nd Floor, “DARE House”, 2, N.S.C. Bose Road, Chennai – \d+\d+\d+ \d+\d+\d+.",
        r"Toll free: 1800 208 9100, T: +91 (0) \d+\d+ \d+\d+\d+\d+ \d+\d+\d+\d+, F: +\d+\d+ (0) \d+\d+ \d+\d+\d+\d+ \d+\d+\d+\d+",
        r"CHOTGDP23004V012223",
        r"Policy Wordings",
        r"Page \d+ of \d+",
    ],
    "doc4.pdf": [
        r"Well Baby Well Mother- Add On Wordings", 
        r"Add On Wordings- Well Baby Well Mother",
        r"Base Product UIN: EDLHLGP21462V032021",
        r"Add On UIN: EDLHLGA23009V012223",
        r"Edelweiss General Insurance Company Limited,",
        r"Corporate Office: 5th Floor, Tower 3, Kohinoor City Mall, Kohinoor City, Kirol Road, Kurla (West), Mumbai - 400 070, Registered Office: ",
        r"Edelweiss House, Off CST Road, Kalina, Mumbai -400 098, IRDAI Regn. No.: 159, CIN: U66000MH2016PLC273758, Reach us on: 1800 12000, Email: support@edelweissinsurance.com, Website: www.edelweissinsurance.com, Issuing/Corporate Office: +91 22 4272 2200, Grievance Redressal Officer: +91 22 4931 4422, Dedicated Toll-Free Number for Grievance: 1800 120 216216.",
        r"Trade logo displayed above belongs to Edelweiss Financial Services Limited and is used by Edelweiss General Insurance Company Limited under license.",
        r"Insurance is the subject matter of solicitation.",
    ],
    "doc5.pdf": [
        r"HDFC ERGO General Insurance Company Limited",                   
        r". IRDAI Reg. No.146. CIN: U66030MH2007PLC177117.",
        r"Registered & ",
        r" Corporate Office: 1st Floor, HDFC House, 165-166 Backbay Reclamation, H. T. Parekh Marg, Churchgate, Mumbai – 400 020.",
        r"Trade Logo displayed above belongs to HDFC Ltd and ERGO International AG",
        r"and used by the Company under license.",
        r"Easy Health UIN: HDFHLIP23024V072223 ",
        r"\d+ | P a g e ",
    ],
    "doc6.pdf": [
        r"For Buy/ Renew/ Service/ Claim related queries Log on to www.icicilombard.com or call 1800 2666",
        r"UIN : ICIHLIP22012V012223",
        r"Golden Shield",                   
        r"CHOTGDP23004V012223 Policy Wordings",
        r"\d+ CIN: L67200MH2000PLC129408",
        r"CIN: L67200MH2000PLC129408",
        r"UIN : ICIHLIP22012V012223 ",
    ],
    "doc7.pdf": [
        r"UIN: NICHLIP25041V022425", 
        r"CIN - U10200WB1906GOI001713",
        r"Regd. & Head Office: Premises No. 18-0374,",
        r"Plot no. CBD-81, New Town, Kolkata - 700156",
        r"Page\s+\d+\s+of\s+\d+",
        r"Arogya Sanjeevani Policy - National ",
    ],
    "doc8.pdf": [
        r"SBI General Insurance Company",
        r"SBI General Insurance Company Limited", 
        r"Limited",                  
        r"Corporate & Registered Office:",
        r"'Natraj',\s*301,\s*Junction of Western Express Highway\s*&\s*Andheri\s*-\s*Kurla Road,\s*Andheri\s*\(E\),\s*Mumbai\s*-\s*400\s*069",
        r"CIN: U66000MH2009PLC190546 I ",
        r"Tel\.\:\s*\+91\s*22\s*42412000\s*I\s*",
        r"www.sbigeneral.in I Logo displayed belongs to State Bank of India and is used by SBI General Insurance Co. Ltd. under license I IRDAI Registration Number 144 I Product Name: Health Insurance Policy Retail.",
        r"UIN\:\s*SBIHLIP11002V021011\s*I\s*IRDAI\s*Reg\s*No\.144",
        r"IRDAI Reg No.144",
    ],
    "docx12.docx": [
        r"SAFEWAY INSURANCE TPA PVT LTD",                   
        r"Cont………",
    ],
    "docx13.docx": [
        r"Postal Life Insurance",
        r"Terms of Contract ",                   
    ],
    
    # Add more files and patterns as needed
}

# Function to clean text using regex patterns
def clean_text(text, patterns):
    for pattern in patterns:
        text = re.sub(pattern, '', text)
    # Also remove extra newlines/whitespace
    text = re.sub(r'\n{2,}', '\n\n', text)  # compress multiple newlines
    return text.strip()

# Apply cleaning
cleaned_data = []
for doc in data:
    file = doc["file_name"]
    content = doc["content"]
    patterns = cleaning_rules.get(file, [])
    cleaned = clean_text(content, patterns)
    cleaned_data.append({
        "file_name": file,
        "cleaned_content": cleaned
    })

# Save cleaned output
with open("cleaned_text.json", "w", encoding="utf-8") as f:
    json.dump(cleaned_data, f, indent=4)

print("✅ Headers/footers removed. Saved to cleaned_text.json.")


✅ Headers/footers removed. Saved to cleaned_text.json.


In [ ]:
for doc in data:
    if doc["file_name"] == "docx13.docx":
        patterns = cleaning_rules["docx13.docx"]
        cleaned = clean_text(doc["content"], patterns)
        print("🔍 Before:\n", doc["content"][:1000])
        print("\n✅ After:\n", cleaned[:1000])


🔍 Before:
 Postal Life Insurance
Terms of Contract
APPLICATION OF RULES: This policy is issued subject to the provisions in the Post Office Life Insurance Rules-2011 and any amendments made to the said Rules from time to time.
PROOF OF AGE:
The onus of providing correct age will rest with insured. Insured person should provide such proof of age so that premium can be based on correct age.
In the event it is established that the age advised by insured was higher than the correct age of the insured i.e. he was paying premium higher than the premium warranted by his/ her correct age, he/ she may be allowed to pay correct premium from current date. However, no refund will be made for any excess premium already paid.
In the event an insured advised a lower age than his correct age and was paying a premium lower than that required for continuance of the policy depending on his
correct age than such policy will be declared to be void and no benefit will be payable under this policy. However, 

## Chunking into Paragraphs + Metadata

In [ ]:
# deleted
import json
import os

def get_overlapping_chunks(paragraphs, window=2, stride=1):
    chunks = []
    for i in range(0, len(paragraphs) - window + 1, stride):
        chunk = "\n\n".join(paragraphs[i:i+window])
        chunks.append(chunk.strip())
    return chunks

# Load the cleaned text file (assumed to be in JSON format)
with open("cleaned_text.json", "r", encoding="utf-8") as f:
    cleaned_docs = json.load(f)

# Store the final chunks with metadata
chunked_data = []

# Parameters
window_size = 2   # Number of paragraphs per chunk
stride = 1        # Overlap step

for doc in cleaned_docs:
    file_name = doc["file_name"]
    content = doc["cleaned_content"]
    
    # Split into paragraphs
    paragraphs = [p.strip() for p in content.split("\n\n") if len(p.strip()) > 30]

    # Create overlapping chunks
    chunks = get_overlapping_chunks(paragraphs, window=window_size, stride=stride)

    for idx, chunk in enumerate(chunks, 1):
        chunked_data.append({
            "file_name": file_name,
            "chunk_id": f"{file_name.replace('.pdf','')}-C{idx}",
            "text": chunk,
            "section": None  # You can annotate later if needed
        })

# Save chunks to JSON
with open("chunked_documents.json", "w", encoding="utf-8") as f:
    json.dump(chunked_data, f, indent=4)

print(f"✅ Chunking complete: {len(chunked_data)} chunks saved.")


✅ Chunking complete: 193 chunks saved.


In [ ]:
import json
import re

def get_word_chunks(text, chunk_size=200, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        if len(chunk.strip()) > 0:
            chunks.append(chunk.strip())
    return chunks

# Load cleaned JSON
with open("cleaned_text.json", "r", encoding="utf-8") as f:
    cleaned_data = json.load(f)

chunked_data = []
chunk_size = 200     # number of words
overlap_size = 50    # overlapping words

for doc in cleaned_data:
    file_name = doc["file_name"]
    content = doc["cleaned_content"]  # or use doc["content"] if that's the key
    
    # Optional: remove extra spaces/newlines
    content = re.sub(r'\s+', ' ', content).strip()
    
    # Generate chunks
    chunks = get_word_chunks(content, chunk_size=chunk_size, overlap=overlap_size)

    for idx, chunk in enumerate(chunks, 1):
        chunked_data.append({
            "file_name": file_name,
            "chunk_id": f"{file_name.replace('.pdf','')}-C{idx}",
            "text": chunk,
            "section": None
        })

# Save to JSON
with open("chunked_documents.json", "w", encoding="utf-8") as f:
    json.dump(chunked_data, f, indent=4)

print(f"✅ Word-based overlapping chunking complete: {len(chunked_data)} chunks saved.")


✅ Word-based overlapping chunking complete: 1095 chunks saved.
